In [44]:
# import h5 reader
import h5py
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

work_path = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/bar_shearwave/quad4/"
h5_path = work_path + "output/fault/"

In [64]:
# Class RunABatch
class RunABatch:
    # Constructor
    def __init__(self, input_set, work_path, FourierTerms = 16):
        # Initialize data paths and batch parameters
        self.work_path = work_path
        self.input_set = input_set
        self.h5_path = work_path + "output/fault/"
        self.FourierTerms = FourierTerms
        
        # Store all cases in the h5_path
        self.existingCasesFile = self.h5_path + "CaseList.csv"
        
        # Flag for whether cases have been run
        self.casesExcuted = False
        
        # Get existing cases
        self.getExistingCasesOfInputSet()
        
        # Get input_set_toRun
        self.input_set_toRun = list(set(input_set) - self.existingCases)
        
        # Run cases
        self.runCases(self.input_set_toRun)
        
        # Get obsevations
        self.Observations = self.getObservations(self.input_set)
    
    # Inline function gets [A, B] list
    def getAAndB(self, fileName):
        A_idx = fileName.find('A')
        B_idx = fileName.find('B')
        slash_idx = fileName.find('-')
        
        # Change this part !! Before applying to new name convention
        A = float(fileName[A_idx + 1 : B_idx - 1])
        B = float(fileName[B_idx + 1 : slash_idx])
        return (A, B)
    
    
    # Function get_existing_files_set
    def getExistingCasesOfInputSet(self):
        # Get all .h5 file names as a list
        myPath = self.h5_path
        onlyFiles = [f for f in listdir(myPath) if (isfile(join(myPath, f)) and f[-3 : ] == '.h5')]
        self.existingCases = set([self.getAAndB(f) for f in onlyFiles])
        
    # Function runCases
    def runCases(self, input_set):
        shell_path = self.work_path + "RunJobsJP.sh"
        shellRead = open(shell_path, 'r')
        list_of_lines = shellRead.readlines()
        shellRead.close()

        AA = [ele[0] for ele in input_set]
        BB = [ele[1] for ele in input_set]

        list_of_lines[9] = "AA=" + str(tuple(AA)).replace(',', '') + "\n"
        list_of_lines[10] = "BB=" + str(tuple(BB)).replace(',', '') + "\n"

        shellWrite = open(shell_path, 'w')
        shellWrite.writelines(list_of_lines)
        shellWrite.close()

        # Run the cases
        !source $shell_path
        
        self.casesExcuted = True
        
        # Return from shell
        return
    
    
    # Function getObservations for the input_set
    def getObservations(self, input_set):
        # Initialize Observations
        Observations = []
        
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.h5_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) + "-fault.h5"
            f = h5py.File(h5_file, 'r')

            # Get time
            time = np.array(f['time']).reshape([-1])
            time = time - np.min(time)
            nOfTSteps = time.shape[0]

            # Get Slip rates
            SlipRates = np.array(f['vertex_fields']['slip_rate'])
            Vx = SlipRates[:, :, 0].transpose()
            Vy = SlipRates[:, :, 1].transpose()
            nOfNodes = Vx.shape[0]

            # Find the Fourier coefficients
            FourierTerms = self.FourierTerms
            T = np.max(time)

            # Compute the Fourier terms
            Ks = np.array(range(FourierTerms))
            coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * time)
            VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in Vx], 0)

            # Compute the fourier coefficients
            # print('time.shape: ', time.shape)
            observation = np.trapz(VxcoskPiTt, x=time)

            # Append the result from this file
            Observations.append(observation)
        
        Observations = np.array(Observations)
        return Observations
    
    

In [72]:
# Input example of getting observations
AA = np.linspace(0.002, 0.014, 7)
BB = AA + 0.004

# Get input set
input_set = set([(AA[i], BB[i]) for i in range(len(AA))])

# Test class RunABatch
myBatch = RunABatch(input_set, work_path)

/home/shengduo/InverseProblems/GPRWorkingField


In [79]:
np.array(list(myBatch.input_set))

array([[0.004, 0.008],
       [0.014, 0.018],
       [0.006, 0.01 ],
       [0.01 , 0.014],
       [0.002, 0.006],
       [0.008, 0.012],
       [0.012, 0.016]])

# Sampling process

In [127]:
# Import the sampling packages
import emcee

# Set up a GP
myGP = GP_predictor(myBatch.input_set, myBatch.Observations)

# Define the log probability function
def log_prob(u, u_low, u_high, y, GP, si_eta = 1.):
    normal_idx = np.all(np.concatenate([u >= u_low, u <= u_high], axis = 1), axis = 1)
    res = - 0.5 / si_eta ** 2 * np.sum((y - GP.predict(u)) ** 2, axis = 1)
    res[~normal_idx] = -np.inf
    return res

# Use a sample y
y = myBatch.Observations[0]

# Sample from p(u|y)
uLow = np.array([0.002, 0.006])
uLength = np.array([0.01, 0.01])
uHigh = uLow + uLength

ndim, nwalkers = 2, 100
p0 = np.random.uniform(size = [nwalkers, ndim]) * uLength + uSt

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[uLow, uHigh, y, myGP], vectorize = True)
sampler.run_mcmc(p0, 1000)

samples = sampler.get_last_sample().coords
samples.shape

(100, 2)

In [133]:
a = np.random.uniform(size=[5, 2])
len(a)

5

# Class BayersianInv

In [153]:
# Define Bayersian Inv that solves a Bayersian inversion problem
class BayersianInv:
    # Constructor
    def __init__(self, u_low, u_high, y, work_path, atol = 1.0e-4, n_samples = 20, MCMCsteps = 1000):
        self.u_low = u_low
        self.u_high = u_high
        self.input_dim = len(u_low)
        
        self.y = y
        self.output_dim = len(y)
        self.work_path = work_path
        self.atol = atol
        
        self.n_samples = n_samples
        self.MCMCsteps = MCMCsteps
        
        self.GPs = []
        self.U = np.empty([0, self.input_dim])
        self.Observations = np.empty([0, self.output_dim])
        self.iterations = 0
    
    # Get the accumulative probability function
    def log_prob(u, y, si_eta = 1.):
        # Apply hard constraints
        normal_idx = np.all(np.concatenate([u >= self.u_low, u <= self.u_high], axis = 1), axis = 1)
        
        # First prior is uniform distribution
        res = np.ones(len(u))
        res[~normal_idx] = -np.inf
        
        # Add all posteriors in each iteration
        for i in range(self.iterations):
            res[normal_idx] += -0.5 / si_eta ** 2 * (
                np.sum(
                    (y - self.GPs[i].predict(u[normal_idx])) ** 2, 
                    axis = 1
                )
            )
        
        # Return the log_probability at the current iteration
        return res
    
    # Run one iteration
    def runOneIteration(n_samples):
        # Sample from the current distribution for u and get the current observations
        sampler = emcee.EnsembleSampler(n_samples, 
                                        self.input_dim, 
                                        self.log_prob, args=[y], 
                                        vectorize = True)
        
        # Initialize uniformly as the starting point
        p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low
        
        # Get the result
        sampler.run_mcmc(p0, self.MCMCsteps)
        self.samples = sampler.get_last_sample().coords
        
        # Check if the standard deviation of the samples is smaller than atol
        if np.std(self.samples) < self.atol:
            # Converging flag True
            return True
        
        else:
            # Run the forward map with the samples
            myBatch = RunABatch(samples, self.work_path)
            self.U = np.append(self.U, samples, axis = 0)
            self.O = np.append(self.O, myBatch.Observations, axis = 0)

            # Get a new Gaussian process
            myGP = GP_predictor(self.U, self.O)
            self.GPs.append(myGP)
            self.iterations += 1
            
            # Converging flag False
            return False

In [105]:
samples = sampler.get_last_sample().coords
samples.shape

(100, 2)

In [147]:
O = np.empty([0, 2])
a = np.random.uniform(size = [5, 2])
O = np.append(O, a, axis = 0)
O

array([[0.64600015, 0.40125205],
       [0.4025001 , 0.93104859],
       [0.76505991, 0.47248174],
       [0.54590508, 0.8524087 ],
       [0.2984607 , 0.41978312]])

In [149]:
np.std(O)

2.041407644915402e-01

In [150]:
O

array([[0.64600015, 0.40125205],
       [0.4025001 , 0.93104859],
       [0.76505991, 0.47248174],
       [0.54590508, 0.8524087 ],
       [0.2984607 , 0.41978312]])

In [75]:
a = np.random.uniform()

# Gaussian predictor

In [88]:
# Import Gaussian-regression related functions
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Pre-process the data
from sklearn import preprocessing

# function train_GP
class GP_predictor:
    # Constructor
    def __init__(self, 
                 input_set, 
                 observation_set, 
                 GPkernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1e2)), 
                 n_optimizers = 9):
        # Scale input data
        self.input_set = [list(x) for x in input_set]
        self.observation_set = [list(x) for x in observation_set]
        self.input_dimension = len(self.input_set[0])
        self.observation_dimension = len(self.observation_set[0])
        self.trainset_length = len(self.input_set)
        
        self.input_scaler = preprocessing.StandardScaler()
        self.input_scaler.fit(np.array(self.input_set))
        
        # Scale output data
        self.observation_scaler = preprocessing.StandardScaler()
        self.observation_scaler.fit(np.array(observation_set))
        
        # Fit Gaussian process
        self.GP = GaussianProcessRegressor(kernel = GPkernel, n_restarts_optimizer = n_optimizers)
        self.GP.fit(self.input_scaler.transform(np.array(self.input_set)), 
                    self.observation_scaler.transform(np.array(self.observation_set)))
        
    # Predict on a new input set
    def predict(self, new_input_set):
        # Predict new observation
        new_observation = self.observation_scaler.inverse_transform(
            self.GP.predict(
                self.input_scaler.transform(np.array(list(new_input_set)).reshape([-1, self.input_dimension]))
            )
        )
        
        return new_observation
        
    

In [69]:
# Generate a set, run cases, get observations

In [ ]:
myGP = GP_predictor(input_set, )